# Simulating Quantum #
by Vy La, Thien Kim Le, and Yusaku Tanaka

***Importing necessary packages***

In [2]:
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib import pyplot as plt
import matplotlib.animation as animation

# Finite Difference Method #
To simulate anything quantum, it is unecessary to find the infinite continuous case so instead an approximate solution is appropriate here. Let's first state the 1-D Schrodinger Equation
$$
-\frac{\hbar^2}{2m}\frac{d^2 \psi}{d x^2} + V\psi = i\hbar\frac{d}{dt}\psi
$$
This is basically the quantum equivalent of Newton's second law where given a set of known initial conditions, Newton's second law makes a mathematical prediction as to what path a given physical system will take over time while the Schrodinger equation  gives the evolution over time of a wave function $\psi$, the quantum-mechanical characterization of an isolated physical system. To discretize the equation, let's rewrite the equation  to the following
$$
\frac{i}{2m}\frac{d^2}{dx^2}\psi_j - iV\psi_j = \frac{d}{dt}\psi_j
$$
where the planks constant $\hbar$ is dropped for simplicity. Using the [finite difference method](https://medium.com/modern-physics/finite-difference-solution-of-the-schrodinger-equation-c49039d161a8), our spatial derivative $\frac{d^2}{dx^2}\psi$ becomes
$$
\frac{d^2}{dx^2}\psi \rightarrow \frac{1}{\Delta x}(\psi_{j-1} - 2\psi_j + \psi_{j+1})
$$
where $\Delta x$ is the stepsize. We'll drop it for simplicity again. Reffering back to our rewritten Schrodinger Equation, we get
$$
\frac{i}{2m}(\psi_{j-1} - 2\psi_j + \psi_{j+1}) - iV\psi_j = \frac{d}{dt}\psi_j
$$
and thus our discrete Schroinger Equation has been found.

All the discrete wave functions used here will be sampled at the same spatial point $x$.

In [3]:
x = np.linspace(-10,10,5000)
deltax = x[1]-x[0]

Something that a wave function must satsify is that they must be normalized. Lets say we have three babies with three different ages. Each of them have a 1 in 3 chance of being that age. Since we have three 1 in 3 chances, if we add them up, it will equal to one. 
$$
\frac{1}{3}+\frac{1}{3}+\frac{1}{3} = 1
$$
To normalize a wave function, it's typically written as the following 
$$
\int_{-\infty}^{\infty} |\psi|^2 \, dx = 1
$$
We're squaring it because the wave function has no physical interpretation meaning it's imaginary, but once we do, the wave function becomes real. Since we're doing a discrete case, we would use the sum instead and utilize our step size.

In [4]:
def norm(phi):
    norm = np.sum(np.square(np.abs(phi)))*deltax
    return phi/np.sqrt(norm)

It is necessary to plot both the real and imaginary representation of the wave function. If you input euler's formula 
$$
e^{ix} = \cos x + i\sin x
$$
into desmo, you would get 2 graphs instead of expecting 1. One of the graph is a sine wave and the other is a consine wave. The reason for producing two 2 graphs is because of the imaginary number. You can get 1 graph, a circle, in a real vs imaginary coordinate plane, but desmo is in the xy-plane so it outputted 2 graphs. Here's an excellent [video](https://youtu.be/cUzklzVXJwo?t=1163) demonstrating a better visualization of this fact.

In [5]:
def complex_plot(x,y,prob=True,**kwargs):
    real = np.real(y)
    imag = np.imag(y)
    a,*_ = plt.plot(x,real,label='Re',**kwargs)
    b,*_ = plt.plot(x,imag,label='Im',**kwargs)
    plt.xlim(-2,2)
    if prob:
        p,*_ = plt.plot(x,np.abs(y),label='$\sqrt{P}$')
        return a,b,p
    else:
        return a,b

Below we define a simple wave function with two equations, euler's formula with momentum 
$$
e^{ix} = \cos x + i\sin x
$$
and a part of the normal distribution equation
$$
e^{-(x - \mu)^2/\sigma^2}
$$
This is where we assume that the particle has some sort of normal dsitribution and some kind of momenta. $\sigma$ is represents the size of the distrubtion and $\mu$ is the mean of the position.

In [6]:
def wave_packet(mu=0,mom=0,sigma=0.2):
    return norm(np.exp(-1j*mom*x)*np.exp(-np.square(x-mu)/sigma/sigma,dtype=complex))

The next two cells is just defining our spatial derivative with some boundary conditions imposed and defining the discrete Schrodinger equation.

In [7]:
def d_dxdx(phi,x=x):
    dphi_dxdx = -2*phi
    dphi_dxdx[:-1] += phi[1:]
    dphi_dxdx[1:] += phi[:-1]
    return dphi_dxdx/deltax 

In [8]:
def d_dt(phi,h=1,m=100,V=0):
    return 1j*h/2/m * d_dxdx(phi) - 1j*V*phi/h

# Runge-Kutta Method #
Here we will introduce the [Runge-Kutta Method](https://medium.com/codex/python-and-physics-runge-kutta-method-93d7e2105230). This method is a numerical approximation of ordinary differential equations developed by Carl Runge and Wilhelm Kutta. We basically take four slope values within an arbitrary interval of any graph that does not necessarily fall onto the actual solution. We then average out the slope which gives a pretty nice approximation of the solution. Since we're taking four slope values, we need a fourth order runge-kutta method which is also widely used. 

In [9]:
def rk4(phi, dt, **kwargs):
    k1 = d_dt(phi, **kwargs)
    k2 = d_dt(phi+dt/2*k1, **kwargs)
    k3 = d_dt(phi+dt/2*k2, **kwargs)
    k4 = d_dt(phi+dt*k3, **kwargs)
    return phi + dt/6*(k1+2*k2+2*k3+k4)

Below we take steps of wave functions through time and save them periodically asnd animate them with the next cell.

In [10]:
def simulate(phi_sim, 
             method='rk4', #runge-kutta method
             V=0, #potential energy
             steps=100000, #saves this amount of steps of the simulation which also determines the duration of the simulation
             dt=1e-1, #kind of like determining the video speed in a youtube video, the larger the number, the faster the simulation
             condition=None, 
             normalize=True,
             save_every=100): #this is like frames of the of the simulation where we save the visualization every number of steps
    simulation_steps = [np.copy(phi_sim)]
    for i in range(steps):
        if method == 'rk4':
            phi_sim = rk4(phi_sim,dt,V=V)
        else:
            raise Exception(f'Unknown method {method}')
        if condition:
            phi_sim = condition(phi_sim)
        if normalize:
            phi_sim = norm(phi_sim)
        if save_every is not None and (i+1) % save_every == 0:
            simulation_steps.append(np.copy(phi_sim))
    return simulation_steps

In [11]:
def animate(simulation_steps,init_func=None):
    fig = plt.figure()
    re,im,prob = complex_plot(x,simulation_steps[0])
    plt.xlim(-2,2)
    plt.ylim(-2,2)
    if init_func:
        init_func()
    plt.legend()

    def animate(frame):
        prob.set_data((x, np.abs(simulation_steps[frame])))
        re.set_data((x, np.real(simulation_steps[frame])))
        im.set_data((x, np.imag(simulation_steps[frame])))
        return prob,re,im

    anim = animation.FuncAnimation(fig, animate, frames=int(len(simulation_steps)), interval=50)
    plt.close()

    return anim

Below are two different kind of wave packet behaviors. The first plot is defining a potential energy barrier to demonstrate qunatum tunneling while the second plot is a simple wave packet with no momenta and just decaying with time. You can in fact create your own potential energy with any sort of barrier ie. a quadratic. Just simply define a new variable that represents your graph and create a new function to plot it.

In [14]:
barrier_weak_potential = np.where((x>1.4)&(x<1.6),1.05e-2,0)
sim_barrier_mom = simulate(wave_packet(mom=-10, sigma = 0.3),V=barrier_weak_potential,steps=50000,save_every=500)

def barrier_init():
    plt.gcf().axes[0].axvspan(1.4, 1.6, alpha=0.2, color='blue')
    plt.xlim(-2,4)
    plt.ylim(-3,3)
    
writergif = animation.PillowWriter(fps=30) 

In [15]:
plot1 = animate(sim_barrier_mom,init_func=barrier_init)
plot1.save('barrier.gif', writer=writergif)

In [15]:
sim_plot2 = simulate(wave_packet(), steps = 200000, save_every = 1000)
plot2 = animate(sim_plot2)
plot2.save('simple.gif', writer=writergif)

C:\Users\vyla5\anaconda3\lib\site-packages\matplotlib\animation.py:887: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you have outputted the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(
